In [23]:
import os
import sys
sys.path.append('..')
import lib.snowpark_conn as spn
import lib.snowpark_runner as spr
import lib.file_helper as fh
import lib.aws_helper as aws

from dotenv import load_dotenv
from datetime import datetime

In [15]:
# IF RUNNING ON AWS Sagemaker:

# aws_helper = aws.AWSHelper(region = 'us-east-2')
# parameter_names = ['uvarnd-snowpark-username', 'uvarnd-snowpark-password']
# parameters = aws_helper.get_parameters(parameter_names)

# USER = parameters['uvarnd-snowpark-username']
# PASSWORD = parameters['uvarnd-snowpark-password']

# IF RUNNING ON LOCAL:
# Change depending on enivornment variable names
load_dotenv()

USER = os.getenv('SP_USER')
PASSWORD = os.getenv('PASSWORD')

In [17]:
# Create Snowpark Connection
sp_conn = spn.SnowparkConnector(
    ACCOUNT = 'hum.us-east-2.aws',
    USER = USER,
    PASSWORD = PASSWORD,
    ROLE = 'RUP',
    WAREHOUSE = 'UVARND_WH',
    DATABASE = 'CORE',
    SCHEMA = 'CLIENT'
)

In [18]:
# Create Snowpark Runner
sp_runner = spr.SnowparkRunner(sp_conn)

In [19]:
THRESHOLD = 16

classification_data = sp_runner.query_classification_dataset(THRESHOLD)

Querying from Snowpark...
Snowpark query done


In [20]:
classification_data

,PROFILE_ID,REACHED_16_EVENTS,RECENT_LAST_EVENT,EVENT_CYCLES_F16,DISTINCT_ARTICLES_F16,PERCENT_GOOGLE_ARTICLES_F16,PERCENT_ARTICLE_CONTENT_F16,AVERAGE_CONTENT_SCORE_F16,DAYS_TO_16_EVENTS,EVENTS,FIRST_EVENT_TIME,EVENT_TIME_16,DISTINCT_DAYS_F16
0,--oINYIBUgM-564PCT_X,1,0,1,2,1.000000,1.000000,0.00,1,16,2022-07-25 11:04:56.852977,2022-07-26 05:58:49.636661,2
1,--0p74IB28LFeUkXSg6b,1,0,2,2,0.000000,1.000000,0.00,133,16,2022-08-30 14:30:38.171966,2023-01-10 16:43:55.720219,2
2,--7egIIBGoM3uBpFKICG,1,0,1,4,1.000000,1.000000,0.00,5,63,2022-08-09 04:30:20.132357,2022-08-14 21:53:19.151456,3
3,--VwtYUBwWEOklUH0r7u,1,0,3,4,0.750000,1.000000,0.00,18,16,2023-01-15 12:39:08.498457,2023-02-02 16:38:58.476149,4
4,--iEtIMBGoM3uBpFEpsG,1,0,4,3,0.000000,1.000000,0.00,109,16,2022-10-07 22:14:59.731847,2023-01-24 19:39:01.060469,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
141784,zxtxKYQBwWEOklUHY9No,1,0,1,2,0.000000,1.000000,0.00,0,52,2022-10-30 15:10:08.024318,2022-10-30 15:21:27.509941,1
141785,zyl3_oEBuqp2E9ofVDxM,1,0,3,1,0.000000,1.000000,0.00,18,16,2022-07-14 20:47:22.374402,2022-08-01 15:55:51.194533,4
141786,zw0BoIYBwWEOklUHVSd-,1,1,1,1,0.000000,1.000000,0.00,8,21,2023-03-02 01:48:07.933678,2023-03-10 05:48:57.620802,4
141787,zxd8hoIBGoM3uBpF1MRU,1,0,2,2,0.500000,0.222222,23153.25,64,55,2022-08-10 06:41:45.518444,2022-10-13 23:11:26.568897,2


In [21]:
clustering_data = sp_runner.query_clustering_dataset(THRESHOLD)

Querying from Snowpark...
Snowpark query done


In [22]:
clustering_data

,PROFILE_ID,REACHED_16_EVENTS,RECENT_LAST_EVENT,EVENT_CYCLES_ALL,DISTINCT_ARTICLES_ALL,PERCENT_GOOGLE_ARTICLES_ALL,PERCENT_ARTICLE_CONTENT_ALL,AVERAGE_CONTENT_SCORE_ALL,DAYS_TO_16_EVENTS,EVENTS,FIRST_EVENT_TIME,LATEST_EVENT_TIME,DISTINCT_DAYS
0,--LbkoEBlNqnsgIRWsTf,1,0,2,7,0.142857,0.700000,51700.956522,6.0,23,2022-06-23 23:17:38.984057,2022-06-29 02:26:05.956743,3
1,--rL64YBEEdskq5qsQWD,1,1,1,1,0.000000,1.000000,0.000000,2.0,17,2023-03-16 19:00:40.948221,2023-03-18 20:30:16.795194,2
2,-09Q54EBuqp2E9ofp8ZX,1,0,1,3,0.000000,0.750000,218042.581395,4.0,43,2022-07-10 08:53:56.598671,2022-07-29 07:15:41.734918,17
3,--GZ7IIB28LFeUkXTMrZ,1,0,2,1,0.000000,1.000000,0.000000,7.0,16,2022-08-30 02:34:07.121719,2022-09-06 12:38:50.044481,3
4,--KyPIMBGoM3uBpF1OJ3,1,0,5,5,0.400000,1.000000,0.000000,134.0,18,2022-09-14 15:51:36.779253,2023-01-26 23:35:55.585732,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2471910,n5-JSYEBlNqnsgIRbaUW,0,0,1,1,1.000000,1.000000,0.000000,NaN,3,2022-06-09 17:35:54.035312,2022-06-09 17:52:55.974155,1
2471911,nIJXJIMB28LFeUkXXI2d,0,0,1,1,1.000000,1.000000,0.000000,NaN,1,2022-09-09 22:20:55.044979,2022-09-09 22:20:55.044979,1
2471912,j0D0o4ABlNqnsgIRgyku,0,0,1,0,0.000000,0.000000,347253.000000,NaN,1,2022-05-08 13:55:53.724086,2022-05-08 13:55:53.724086,1
2471913,RULUw4YBwWEOklUHE03-,0,1,1,0,0.000000,0.000000,0.000000,NaN,4,2023-03-09 00:45:03.461484,2023-03-09 00:45:08.526544,1
